<a href="https://colab.research.google.com/github/sgupt1802/MyTorchTuts/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Convert MNIST Image files into tensors of 4 dimensions.


In [3]:
### We will transform the images data because the input will be a two dimensional image but we need a 4 dimensions because we want to store the (number of images, the height, the width and the color)
transform=transforms.ToTensor()


In [4]:
# Train_data
train_data=datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:01<00:00, 6027176.59it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 1084959.19it/s]

Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 9261358.13it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 11510893.52it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
len(train_data)

60000

In [6]:
#Test data
test_data=datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

In [7]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [19]:
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10,shuffle=False)

In [20]:
## Defining the Model
conv1=nn.Conv2d(1,6,3,1)
conv2=nn.Conv2d(6,18,3,1)

In [21]:
#Taking the first MNIST Record
for i,(X_Train,y_train) in enumerate(train_data):
  break

In [23]:
X_Train.shape

torch.Size([1, 28, 28])

In [24]:
x=X_Train.view(1,1,28,28)

In [25]:
## First convolution
x=F.relu(conv1(x))

In [26]:
x.shape

torch.Size([1, 6, 26, 26])

In [27]:
## pass through the pooling layer
x=F.max_pool2d(x,2,2)   # kernel of 2 and the stride of 2

In [28]:
x.shape    ## 26/2=13

torch.Size([1, 6, 13, 13])

In [29]:
## second convolution
x=F.relu(conv2(x))

In [30]:
x.shape

torch.Size([1, 18, 11, 11])

In [31]:
x=F.max_pool2d(x,2,2)

In [32]:
x.shape

torch.Size([1, 18, 5, 5])

### Building the CNN Model

In [35]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(1,6,3,1)
    self.conv2=nn.Conv2d(6,18,3,1)

    ##fully connected layer
    self.fc1=nn.Linear(5*5*18,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  def forward(self,X):
    ## first pass
    X=F.relu(self.conv1(X))
    X=F.max_pool2d(X,2,2)  ##2*2 kernel and stride

    ## second pass
    X=F.relu(self.conv2(X))
    X=F.max_pool2d(X,2,2)  ##2*2 kernel and stride

    ##Review to flatten this out
    X=X.view(-1,18*5*5)                       ## -1 so that we can vary the batch size

    ## Fully connected layers
    X=F.relu(self.fc1(X))
    X=F.relu(self.fc2(X))
    X=self.fc3(X)

    return F.log_softmax(X,dim=1)

In [37]:
##Create an instance of model
torch.manual_seed(42)
model=ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 18, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=450, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [38]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)